In [ ]:
!pip install -Uq openai-agents

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)

In [5]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [6]:
from agents import set_default_openai_client, set_tracing_disabled

set_default_openai_client(external_client)
set_tracing_disabled(True)

In [8]:
import asyncio
import random
from typing import Any

from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, Usage, function_tool

### Basic

In [9]:
class TestHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0
        self.name = "TestHooks"

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### {self.name} {self.event_counter}: Agent {agent.name} started. Usage: {context.usage}")

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(f"### {self.name} {self.event_counter}: Agent {agent.name} ended. Usage: {context.usage}, Output: {output}")

In [13]:
start_hook = TestHooks()

starting_agent = Agent(
    name="Email Fraud Detection Agent",
    instructions=(
        "You are an email fraud detection agent. Analyze incoming email content "
        "and flag any messages that appear to be phishing, scams, or fraudulent financial activity."
    ),
    model=model
)

async def main():
    result = await Runner.run(
        starting_agent,
        hooks=start_hook,
        input="<email>Dear user, your bank account has been compromised. Please click here to verify your details.</email>"
    )

    print(result.final_output)

asyncio.run(main())

### TestHooks 1: Agent Email Fraud Detection Agent started. Usage: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
### TestHooks 2: Agent Email Fraud Detection Agent ended. Usage: Usage(requests=1, input_tokens=52, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=309, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=361), Output: This email is highly suspicious and likely a phishing attempt. Here's why:

*   **Generic Greeting:** The email starts with "Dear user," which is a common tactic in phishing emails. Legitimate banks usually address you by your name.
*   **Sense of Urgency:** The message creates a sense of urgency by claiming the account has been "compromised." This is intended to scare the recipient into acting quickly without thinking.
*   **Suspicious Link:** The instruction to "click her

### Advance

In [18]:
class ExampleHooks(RunHooks):
    def __init__(self):
       self.event_counter = 0

    def _usage_to_str(self, usage: Usage) -> str:
        return f"{usage.requests} requests, {usage.input_tokens} input tokens, {usage.output_tokens} output tokens, {usage.total_tokens} total tokens"

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Agent {agent.name} started. Usage: {self._usage_to_str(context.usage)}"
        )

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Agent {agent.name} ended with output {output}. Usage: {self._usage_to_str(context.usage)}"
        )

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Tool {tool.name} started. Usage: {self._usage_to_str(context.usage)}"
        )

    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Tool {tool.name} ended with result {result}. Usage: {self._usage_to_str(context.usage)}"
        )

    async def on_handoff(
        self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent
    ) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Handoff from {from_agent.name} to {to_agent.name}. Usage: {self._usage_to_str(context.usage)}"
        )

In [21]:
hooks = ExampleHooks()

@function_tool("get_temperature")
def get_temperature(max_temp: int) -> int:
    """Simulate checking the current temperature (in Celsius)."""
    return random.randint(0, max_temp)

# def get_temperature(max: int) -> int:
#     return max  # always return max to simulate edge case

@function_tool("heat_advice")
def heat_advice(temp: int) -> str:
    """Provide health advice based on a high temperature."""
    return (
        f"It's {temp}°C. This is quite hot. Stay hydrated, avoid outdoor activities, "
        "and wear light clothing."
    )


heat_agent = Agent(
    name="Heat Advisory Agent",
    instructions="Provide advice on how to deal with hot weather conditions.",
    tools=[heat_advice],
    model=model,
)

starting_agent = Agent(
    name="Weather Monitor Agent",
    instructions=(
        "Generate a simulated temperature between 0 and a user-specified max. "
        "If temperature > 30°C, hand off to the heat advisory agent. "
        "Otherwise, return the temperature with a message: 'No heat advisory needed.'"
    ),
    tools=[get_temperature],
    handoffs=[heat_agent],
    model=model,
)


async def main() -> None:
    user_input = input("Enter the maximum temperature (°C) to simulate: ")
    ans = await Runner.run(
        starting_agent,
        hooks=hooks,
        input=f"Simulate a temperature between 0 and {user_input}.",
    )

    print(ans.final_output)

asyncio.run(main())

Enter the maximum temperature (°C) to simulate: 45
### 1: Agent Weather Monitor Agent started. Usage: 0 requests, 0 input tokens, 0 output tokens, 0 total tokens
### 2: Tool get_temperature started. Usage: 1 requests, 103 input tokens, 7 output tokens, 110 total tokens
### 3: Tool get_temperature ended with result 9. Usage: 1 requests, 103 input tokens, 7 output tokens, 110 total tokens
### 4: Agent Weather Monitor Agent ended with output The temperature is 9°C. No heat advisory needed.
. Usage: 2 requests, 218 input tokens, 21 output tokens, 239 total tokens
The temperature is 9°C. No heat advisory needed.

